<span style='color:#009999'> <span style='font-family:serif'> <font size="15"> **ECCO from NASA earth Data Cloud**<span style='color:#0066cc'> 

<img src="img/ECCOv4.png" alt="drawing" width="750"/>    


<span style='color:#0066cc'><font size="5"> **About the Estimating the Circulation and Climate of the Ocean" [ECCO](https://ecco-group.org/) consortium**
1. <font size="3"><span style='color:Black'> Uses a combination of assimilated data from (various sources) to constrain the model.
2. <font size="3"><span style='color:Black'> The native grid is that of a Cube Sphere, which results in data that has **complex topology**



<span style='color:#ff6666'><font size="5">Requirements
1. <font size="3"><span style='color:Black'> Have a Bearer Token for EarthData in the Cloud (See Getting Started Notebook).
2. <font size="3"><span style='color:Black'> Upload the Bearer Token from local file`token.json`



 <span style='color:#ff6666'><font size="5">Objectives
- <font size="3"><span style='color:Black'> Understand `Dimensions`, `Groups`, `Maps` and `Arrays` in the DAP4 model.
- <font size="3"><span style='color:Black'> Develop understanding of metadata of remote files accessed via OPeNDAP.
- <font size="3"><span style='color:Black'> How to integrate a workflow involving multiple OPeNDAP URLs and xarray parallelism, lazy evaluation, and
- <font size="3"><span style='color:Black'> To demonstrate an advanced workflow for remote access and plotting of **Level 4** with complex Topology ECCOv4 Data via OPeNDAP



<span style='color:#ff6666'><font size="5"> Browsing Data:

<font size="3"><span style='color:Black'> Broad information about the dataset can be found in the PODAAC website (see [here](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO))


<font size="3"><span style='color:Black'> Some variables or interest are:

- <font size="3"> [Native grid](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4)
- <font size="3"> [Temperature and Salinity](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_TEMP_SALINITY_LLC0090GRID_MONTHLY_V4R4)
- <font size="3"> [Velocities](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_OCEAN_VEL_LLC0090GRID_MONTHLY_V4R4)
- <font size="3"> [Mixed layer depth](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_MIXED_LAYER_DEPTH_LLC0090GRID_MONTHLY_V4R4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from pydap.client import open_url
import json
import cartopy.crs as ccrs
import dask

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Access EARTHDATA**

<font size="3"><span style='color:Black'> Many of the data variables can be browsed [here](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO). Two types of data can be found:

1. <font size="3"> Interpolated data onto a regular Lat-Lon grid.
2. <font size="3"> Data on the native LLC (lat-lon-cap) grid.

<font size="3"><span style='color:Black'> Here we will work with the original data defined on the Lat-Lon-Cap (LLC90) grid.

In [ ]:
Grid_url = 'dap4://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Geometry%20Parameters%20for%20the%20Lat-Lon-Cap%2090%20(llc90)%20Native%20Model%20Grid%20(Version%204%20Release%204)/granules/GRID_GEOMETRY_ECCO_V4r4_native_llc0090'

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Import Token Authorization and create Session**
 


In [ ]:
# load token json data
with open('token.json', 'r') as fp:
    Authorization = json.load(fp)

# pass Token Authorization to a new Session.
my_session = requests.Session()
my_session.headers = Authorization


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Access Lazy access to data via pydap**

<font size="3"> No data has yet been downloaded

In [ ]:
%%time
ds_grid = open_url(Grid_url, session=my_session, protocol="dap4")

In [ ]:
ds_grid.tree()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot Depth along native grid**

In [ ]:
Depth = ds_grid['Depth'][:]

In [ ]:
Depth.attributes

In [ ]:
Depth.shape, Depth.dimensions

In [ ]:
%%time
Variable = [Depth[i].data for i in range(13)]

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES = [
    axes[4, 0], axes[3, 0], axes[2, 0], axes[4, 1], axes[3, 1], axes[2, 1],
    axes[1, 1], 
    axes[1, 2], axes[1, 3], axes[1, 4], axes[0, 2], axes[0, 3], axes[0, 4],
]
for i in range(len(AXES)):
    AXES[i].contourf(Variable[i], np.linspace(0, 6000, 100), cmap='Greys_r')


for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot another Variable**

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot with corrected Topology**

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES_NR = [
    axes[3, 0], axes[2, 0], axes[1, 0], axes[3, 1], axes[2, 1], axes[1, 1],
]
AXES_CAP = [axes[0, 0]]
AXES_R = [
    axes[1, 2], axes[2, 2], axes[3, 2], axes[1, 3], axes[2, 3], axes[3, 3],
]
for i in range(len(AXES_NR)):
    AXES_NR[i].contourf(Variable[i], np.linspace(0, 6000, 100), cmap='Greys_r')

for i in range(len(AXES_CAP)):
    AXES_CAP[i].contourf(Variable[6].transpose()[:, ::-1], np.linspace(0, 6000, 100), cmap='Greys_r')

for i in range(len(AXES_R)):
    AXES_R[i].contourf(Variable[7+i].transpose()[::-1, :], np.linspace(0, 6000, 100), cmap='Greys_r')


for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plot another Variable**

- <font size="3"> Inspect attributes to understand what these variables
- <font size="3"> Make sure to correct for missing values (`decode`)

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Use xarray to aggregate all Temperature for the entire URLs**



In [ ]:
import xarray as xr

In [ ]:
# Start with a single one
Temp_2017_base = "dap4://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Ocean%20Temperature%20and%20Salinity%20-%20Monthly%20Mean%20llc90%20Grid%20(Version%204%20Release%204)/granules/OCEAN_TEMPERATURE_SALINITY_mon_mean_2017-"
month = '01'
end_ = '_ECCO_V4r4_native_llc0090'
Temp_2017 = Temp_2017_base + month + end_


In [ ]:
temp_ds = open_url(Temp_2017, session=my_session)

In [ ]:
temp_ds.tree()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Add a Constraint expression to the URL to ONLY retrieve THETA**

<font size="3"> You can do that by appending to the <base_URL> the CE syntax:

```python
<base_URL>?dap4.ce=/<VarName>
```

<font size="3"> where `<VarName>` is the name of the Array you want to keep

In [ ]:
%%time
temp_ds = open_url(Temp_2017+'?dap4.ce=/THETA', session=my_session)

In [ ]:
temp_ds.tree()

In [ ]:
temp_ds

In [ ]:
%%time
dataset = xr.open_dataset(Temp_2017, engine='pydap', session=my_session)

In [ ]:
dataset

<font size="5"><span style='color:#0066cc'> **NOTE**<span style='color:black'> with xarray - need to include all dimensions associated with THETA in the constraint expression.


In [ ]:
%%time
CE = '?dap4.ce=/THETA;/SALT;/tile;/j;/k;/i;/time'
dataset = xr.open_dataset(Temp_2017+CE, engine='pydap', session=my_session)

<font size="5"><span style='color:#0066cc'> **Create a list of URLs that spans the entire 2017**<span style='color:black'> 
```python
[f'-{i:02}' for i in range(1, 13)]
```

<font size="3.5"><span style='color:#0066cc'>**NOTE**<font size="3.5"><span style='color:black'>: with xarray - need to include all dimensions associated with THETA in the constraint expression.
:



In [ ]:
Temp_2017 = [Temp_2017_base + f'{i:02}' + end_+CE for i in range(1, 13)]

In [ ]:
%%time
xr.open_mfdataset(Temp_2017, engine='pydap', session=my_session, parallel=True, chunks={'time':1})

<font size="5"><span style='color:#0066cc'>**Exercise**

- <font size="3.5"><span style='color:black'> Create an `xarray.Dataset` for Temperature (`THETA`) and Salinity (`SALT`) for years/months available
- <font size="3.5"><span style='color:black'> Make some surface maps to visualize these variables in the native and corrected topology.